In [1]:
import pandas as pd
from sklearn.feature_selection import mutual_info_classif

In [2]:
x_data = pd.read_csv('RNAseq_with_HGNC_symbols.csv', index_col=0)
x_data.index.name = None
x_data

,100130426,100133144,UBE2Q2P2,HMGB1P1,10431,136542,LOC155060,RNU12-2P,SSX9P,317712,...,ZXDA|7789,ZXDB|158586,ZXDC|79364,ZYG11A|440590,ZYG11B|79699,ZYX|7791,ZZEF1|23140,ZZZ3|26009,psiTPTE22|387590,tAKR|389932
sample_0,0.0,2.017209,3.265527,5.478487,10.431999,0.0,7.175175,0.591871,0.0,0.0,...,4.926711,8.210257,9.723516,7.220030,9.119813,12.003135,9.650743,8.921326,5.286759,0.000000
sample_1,0.0,0.592732,1.588421,7.586157,9.623011,0.0,6.816049,0.000000,0.0,0.0,...,4.593372,7.323865,9.740931,6.256586,8.381612,12.674552,10.517059,9.397854,2.094168,0.000000
sample_2,0.0,3.511759,4.327199,6.881787,9.870730,0.0,6.972130,0.452595,0.0,0.0,...,5.125213,8.127123,10.908640,5.401607,9.911597,9.045255,9.788359,10.090470,1.683023,0.000000
sample_3,0.0,3.663618,4.507649,6.659068,10.196184,0.0,7.843375,0.434882,0.0,0.0,...,6.076566,8.792959,10.141520,8.942805,9.601208,11.392682,9.694814,9.684365,3.292001,0.000000
sample_4,0.0,2.655741,2.821547,6.539454,9.738265,0.0,6.566967,0.360982,0.0,0.0,...,5.996032,8.891425,10.373790,7.181162,9.846910,11.922439,9.217749,9.461191,5.110372,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sample_796,0.0,1.865642,2.718197,7.350099,10.006003,0.0,6.764792,0.496922,0.0,0.0,...,6.088133,9.118313,10.004852,4.484415,9.614701,12.031267,9.813063,10.092770,8.819269,0.000000
sample_797,0.0,3.942955,4.453807,6.346597,10.056868,0.0,7.320331,0.000000,0.0,0.0,...,6.371876,9.623335,9.823921,6.555327,9.064002,11.633422,10.317266,8.745983,9.659081,0.000000
sample_798,0.0,3.249582,3.707492,8.185901,9.504082,0.0,7.536589,1.811101,0.0,0.0,...,5.719386,8.610704,10.485517,3.589763,9.350636,12.180944,10.681194,9.466711,4.677458,0.586693
sample_799,0.0,2.590339,2.787976,7.318624,9.987136,0.0,9.213464,0.000000,0.0,0.0,...,5.785237,8.605387,11.004677,4.745888,9.626383,11.198279,10.335513,10.400581,5.718751,0.000000


In [3]:
y_labels = pd.read_csv('labels.csv')['Class']
y_labels

0      PRAD
1      LUAD
2      PRAD
3      PRAD
4      BRCA
       ... 
796    BRCA
797    LUAD
798    COAD
799    PRAD
800    PRAD
Name: Class, Length: 801, dtype: object

In [4]:
print(y_labels)
class_to_num = {cls: i for i, cls in enumerate(y_labels.unique())}
print(class_to_num)
y_encoded = y_labels.map(class_to_num)
print(y_encoded)
y_encoded_values = y_encoded.values

0      PRAD
1      LUAD
2      PRAD
3      PRAD
4      BRCA
       ... 
796    BRCA
797    LUAD
798    COAD
799    PRAD
800    PRAD
Name: Class, Length: 801, dtype: object
{'PRAD': 0, 'LUAD': 1, 'BRCA': 2, 'KIRC': 3, 'COAD': 4}
0      0
1      1
2      0
3      0
4      2
      ..
796    2
797    1
798    4
799    0
800    0
Name: Class, Length: 801, dtype: int64


In [5]:
X_values = x_data.values
print(X_values)

[[ 0.          2.01720929  3.26552691 ...  8.92132623  5.28675919
   0.        ]
 [ 0.          0.59273209  1.58842082 ...  9.39785429  2.09416849
   0.        ]
 [ 0.          3.51175898  4.32719872 ... 10.09046974  1.68302267
   0.        ]
 ...
 [ 0.          3.24958187  3.70749166 ...  9.46671072  4.6774575
   0.5866927 ]
 [ 0.          2.59033853  2.78797567 ... 10.40058062  5.71875068
   0.        ]
 [ 0.          2.32524248  3.80593214 ...  9.84479363  4.55071601
   0.        ]]


In [6]:
X_values.shape

(801, 20531)

In [7]:
mi_scores = mutual_info_classif(X_values, y_encoded_values, discrete_features=False, n_neighbors=3, random_state=0)

In [8]:
mi_wfeatures = pd.Series(mi_scores, index=x_data.columns)

In [9]:
mi_wfeatures_sorted = mi_wfeatures.sort_values(ascending=False)[:200]

In [10]:
mi_wfeatures_sorted

HNF1B|6928      0.960557
TMPRSS2|7113    0.880003
SPDEF|25803     0.859675
EFHD1|80303     0.830510
TRPS1|7227      0.809857
                  ...   
ZNF552|79818    0.566833
FASN|2194       0.566687
CDX2|1045       0.566619
SEC22C|9117     0.566327
COL4A3|1285     0.566209
Length: 200, dtype: float64

In [11]:
top_200_feature_names = mi_wfeatures_sorted.index
top_200_feature_names

Index(['HNF1B|6928', 'TMPRSS2|7113', 'SPDEF|25803', 'EFHD1|80303',
       'TRPS1|7227', 'GAL3ST1|9514', 'IHH|3549', 'EMX2OS|196047',
       'HOMER2|9455', 'FOXA1|3169',
       ...
       'BAIAP2L1|55971', 'PVRL4|81607', 'SNRPN|6638', 'PDZD3|79849',
       'PDZK1|5174', 'ZNF552|79818', 'FASN|2194', 'CDX2|1045', 'SEC22C|9117',
       'COL4A3|1285'],
      dtype='object', length=200)

In [12]:
x_data_wtop_200_features = x_data[top_200_feature_names]

In [13]:
x_data_wtop_200_features

,HNF1B|6928,TMPRSS2|7113,SPDEF|25803,EFHD1|80303,TRPS1|7227,GAL3ST1|9514,IHH|3549,EMX2OS|196047,HOMER2|9455,FOXA1|3169,...,BAIAP2L1|55971,PVRL4|81607,SNRPN|6638,PDZD3|79849,PDZK1|5174,ZNF552|79818,FASN|2194,CDX2|1045,SEC22C|9117,COL4A3|1285
sample_0,5.248778,15.316681,14.271835,7.974615,7.689285,4.479011,7.455015,5.939029,11.607335,12.537182,...,12.073743,10.231161,13.661210,0.591871,3.478079,8.771998,15.027133,4.852678,10.710927,5.229403
sample_1,3.890826,9.975275,6.543954,3.137061,8.292989,2.094168,0.811142,0.587845,5.593554,10.563120,...,12.853232,9.871788,8.425124,0.323658,1.590818,7.130241,12.397664,0.323658,8.601206,4.776636
sample_2,1.074163,15.387896,13.131507,4.947105,10.833428,4.306933,0.452595,0.000000,10.696611,13.962652,...,10.805478,10.549361,11.359848,1.306846,2.438799,7.521781,16.751583,2.438799,10.093391,1.683023
sample_3,2.650029,15.093566,14.174489,7.073392,9.562801,1.637239,3.006442,0.768587,12.179160,13.702346,...,10.943958,9.903685,12.237123,0.434882,2.058697,9.528165,14.844157,0.434882,11.184931,4.321690
sample_4,0.000000,9.514874,12.263791,12.870781,14.439409,3.208502,1.095654,5.168542,9.072961,13.343505,...,10.864279,11.626804,10.515769,0.360982,5.456977,10.705356,14.408098,0.000000,9.623889,1.831269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sample_796,0.000000,4.090701,11.911017,12.902562,14.381415,0.865919,0.000000,6.335924,8.420962,13.277324,...,9.234166,11.500309,11.002189,1.159435,2.665870,9.013158,13.748727,0.000000,9.463238,0.865919
sample_797,9.488065,10.555299,11.517679,5.679970,5.940820,7.462715,5.342420,0.000000,7.210028,12.299526,...,10.394377,9.961822,10.722466,0.618051,1.650305,7.827838,13.461428,3.334983,8.456424,10.054699
sample_798,7.092662,10.414622,7.044121,9.321513,8.771757,8.192574,9.091367,8.139649,8.347298,7.838107,...,9.834956,6.597641,9.901755,5.634492,6.559773,7.601399,12.580186,10.256716,8.614404,6.887842
sample_799,8.328302,15.250250,13.909096,7.056811,8.947774,3.450934,6.120070,2.991826,11.693696,13.576614,...,11.828482,9.996982,12.049260,2.799357,2.314465,8.496490,13.674468,8.387285,9.554815,6.598079


In [14]:
x_data_wtop_200_features.to_csv("RNAseq_with_HGNC_symbols_top200.csv")